# Train the RPS model

For transfer learning see [transfer learning](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

For saving/loading: [save load run](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html)

## Setup
1. Download data

In [1]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=11IkCeaEsjysSaWgMEI3SwkSzJ1Tmxz1i&confirm=t' -O data.zip
!unzip data.zip
!rm -rf data.zip

zsh:1: command not found: wget
unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


2. Import libraries

In [0]:
import torch
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
import cv2 as cv
import numpy as np

3. Set up device

In [0]:
d = torch.device("cpu")
# Use a CUDA GPU if possible (Apple Silicon MPS backend technically works, but is confusingly slow)
if torch.cuda.is_available():
    d = torch.device("cuda:0")

4. Set up preprocessing

In [14]:
#preprocess = ResNet18_Weights.IMAGENET1K_V1.transforms()
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

5. Load in model

In [5]:
# Load in a pretrained model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.to(d)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
# Load presaved model
model = torch.load("../model.pth")
model.to(d)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Tasks:
Read whole training videos into memory

Read clutter dataset into memory

## Training


### Load in training data
#### 1. Rock

In [25]:
video_file = cv.VideoCapture("../data/rock/rock.mp4")

# Count frames
num_frames = 0
while True:
    ret, frame = video_file.read()
    if not ret:
        break
    num_frames += 1

frames = torch.empty(num_frames,3,224,224)
frames_index = 0
video_file = cv.VideoCapture("../data/rock/rock.mp4")
while True:
    ret, frame = video_file.read()
    if not ret:
        break

    # Rearrange the channels
    image = frame[:,:,[2,1,0]]

    # Run preprocess
    image_tensor = preprocess(image).to(d)

    # Append to processed frames
    frames[frames_index] = image_tensor

print(f'Loaded {num_frames} frames')

Loaded 243 frames


In [5]:
# Save model
torch.save(model, "../model.pth")